In [ ]:
!pip install scipy
!pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt
plt.style.available

In [ ]:
plt.style.use('seaborn-v0_8-white')

In [ ]:
data_path = '/content/drive/MyDrive/ESTUDIO_TIME_SERIES'
#·data_path = 'C:/Users/XXX/ESTUDIO_TIME_SERIES'

# 1. Dataset Preparation


In [ ]:
#from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape, Flatten, GRU
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import re


# df1 = pd.read_csv('cpu_percent_results_2.csv', usecols=['beginTimeSeconds', 'CPU percent'])
# df2 = pd.read_csv('uptime_results_2.csv', usecols=['Uptime'])
# df3 = pd.read_csv('tx_rx_results_2.csv',
#                   usecols=['Rx packets', 'Tx packets'])
# df4 = pd.read_csv('nb_np_results_2.csv',
#                   usecols=['NetBytes In', 'NetBytes Out', 'NetPackets In', 'NetPackets Out'])
# df5 = pd.read_csv('disk_memory_results_2.csv',
#                   usecols=['Disk read/s', 'Disk write/s', 'Free Disk', 'Free Memory', 'Used Disk',
#                            'Used Memory'], dtype='str')
# df6 = pd.read_csv('disk_percent_results_2.csv', usecols=['Disk Used percent'])
# df7 = pd.read_csv('memory_percent_results_2.csv', usecols=['Memory Used percent'])

# df = pd.concat([df1, df2, df3, df4, df5, df6, df7], axis=1)
# df = df2.copy()
# df.head()

In [ ]:
#drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('data.csv', index_col=0, parse_dates=['beginTimeSeconds'])
df.head()

In [ ]:
df.shape

In [ ]:
new_order_columns = [
    'Free Memory',
    'Used Memory',
    'Free Disk',
    'Used Disk',
    'Disk read/s',
    'Disk write/s',
    'NetBytes In',
    'NetBytes Out',
    'NetPackets In',
    'NetPackets Out',
    'Rx packets',
    'Tx packets',
    'CPU percent',
    'Memory Used percent',
    'Disk Used percent',
    'Uptime',
]
df = df[new_order_columns]
df.head()

In [ ]:
def cut_col_name(col_name):
  return re.sub("[^a-zA-Z]", '', col_name)

In [ ]:
## df['beginTimeSeconds'] = pd.to_datetime(df['beginTimeSeconds'], unit='s')
## df = df.set_index('beginTimeSeconds')

In [ ]:
# cols = df.columns

# N = df.shape[0]
# plots = df[['Free Disk', 'Used Disk', 'Disk Used percent']][:N]
# # plots.index = df['beginTimeSeconds'][0:N]
# plots.index = df.index[:N]
# _ = plots.plot(subplots=True, figsize=(25,25))
# plt.legend(fontsize='xx-large')

# # plt.grid(True)
# plt.savefig('dataset.png')

In [ ]:
fsize_labels = 23
fsize_ticks = 25
fsize_title = 23
figsize = (15, 5)

# for col in df.columns:

#   fig, ax = plt.subplots(figsize = figsize)
#   ax.plot(df.index, df[col].values)
#   ax.set_title(col, fontsize=fsize_title)
#   ax.set_xlabel('Time', fontsize=fsize_labels)
#   ax.set_ylabel('Value', fontsize=fsize_labels)
#   ax.tick_params(labelsize=fsize_ticks)
#   ax.grid(False)

#   plt.tight_layout()
#   plt.savefig(f'{data_path}/DATA_COMPLETA/{cut_col_name(col)}_data.jpg')

# plt.show()

## 1.1. Feature management



It is necessary to eliminate a certain segment of the information due to its lack of temporality, since it would generate errors during training.
The information to be predicted corresponds to data collected after July 5, 2023.

In [ ]:
df = df[(df.index <= '2023-07-05 00:00:00')]

In [ ]:
# cols = df.columns

# N = df.shape[0]
# plots = df[cols[-4:]][:N]
# # plots.index = df['beginTimeSeconds'][0:N]
# plots.index = df.index[0:N] # Time Variable
# _ = plots.plot(subplots=True, figsize=(25,25))

In [ ]:
fsize_labels = 23
fsize_ticks = 20
fsize_title = 23

# for col in df.columns:

#   fig, ax = plt.subplots(figsize = figsize)
#   ax.plot(df.index, df[col].values)
#   ax.set_title(col, fontsize=fsize_title)
#   ax.set_xlabel('Time', fontsize=fsize_labels)
#   ax.set_ylabel('Value', fontsize=fsize_labels)
#   ax.tick_params(labelsize=fsize_ticks)
#   ax.grid(False)

#   plt.tight_layout()
#   plt.savefig(f'{data_path}/DATA_LIMITADA/{cut_col_name(col)}_data.jpg')

# plt.show()

## 1.2. Descriptive statistics

In [ ]:
for column in df.columns:
  print(f'\nVariable [{column}]\n', df[column].describe())

In [ ]:
for col in df.columns:
  fig, axe = plt.subplots(figsize=(5, 7))
  box = axe.boxplot(df[col], notch=True, vert=True, autorange=True, sym="")
  for box in box['boxes']:
      box.set_color('b')
      axe.set_title(col, fontsize=25)
      axe.tick_params(labelsize=25)
      plt.tight_layout()
      # plt.savefig(f'{data_path}/DATA_STATISTICS/{cut_col_name(col)}_data.jpg')
plt.show()

## 1.3. Variability determination


In [ ]:
print(len(df.columns))
for column in df.columns:
  arr = df[column].unique()
  print(f'Column: [{column}] :: Unique data [{arr}], Total Unique Data [{len(arr)}]')

### 1.3.3 Correlation matrix

The correlation of variables is identified.
A higher correlation when the values are closer to -1 and 1.
It indicates if the variables have an impact on each other.

In [ ]:
correlation_matrix = df.corr()

print(correlation_matrix)

In [ ]:
import seaborn as sns

plt.figure(figsize=(25, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='Blues', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix')
# plt.savefig(f'/content/drive/MyDrive/ESTUDIO_TIME_SERIES/DATA_STATISTICS/cor_matrix.jpg')
plt.show()

## 1.4. Data Pre-processing


### 1.4.1. Partitioning in Training, Validation and Testing datasets

Unlike other Deep Learning models, in the case of time series **it must be guaranteed that the partitions are generated without randomly mixing the data**.

Note:
- The training set (*train*) will be used to find the model parameters.
- The validation set (*val*) to verify that there is no *under/over-fitting* of the model and to adjust its hyperparameters.
- The test set (*test*) to test the best model found during training/validation.

In this case we will use the same function implemented for the univariate models with the difference that instead of entering a *series* of Pandas, we will enter the complete *DataFrame*.

Therefore, the function will return three *dataframes* (train, val and test):

In [ ]:
fsize_labels = 30
fsize_ticks = 25
fsize_title = 30
figsize = (15, 5)

In [ ]:
datasets = {}
# tr_ptg = 0.7
# vl_ptg = 0.15
# ts_ptg = 0.15
num_folds = 4
cut_dates = {}

for i, fold in enumerate(range(0, num_folds - 1)):
  len_data = len(df) / num_folds
  len_train_data = int(len_data * (i + 1))

  train_data = int(len_train_data * 0.8)
  val_data = int(len_train_data * 0.2)

  print('--' * 20)
  print(f'Training - iteration {i}  has {len_train_data}')
  tr = df[:train_data]
  vl = df[train_data:train_data + val_data]
  ts = df[len_train_data:len_train_data + int(len_data)]

  cut_dates[i] = [tr.index[-1], vl.index[-1], ts.index[-1]]

  print(f'Cut date train dataset: {tr.index[0]} - {tr.index[-1]}')
  print(f'Cut date val dataset: {vl.index[0]} - {vl.index[-1]}')
  print(f'Cut date test dataset: {ts.index[0]} - {ts.index[-1]}')
  print('--' * 20)

  datasets[i] = [tr, vl, ts]

In [ ]:
line_width = 3

In [ ]:
n_fold = 3

# for col in df.columns:
#   for fold in range(n_fold):

#     print(f'Presentando el fold {fold} para {col}')
#     tr_ds = datasets[fold][0]
#     val_ds = datasets[fold][1]
#     ts_ds = datasets[fold][2]

#     fig, ax = plt.subplots(figsize = figsize)

#     ax.plot(tr_ds.index, tr_ds[col].values, linewidth=2)
#     ax.plot(val_ds.index, val_ds[col].values, linewidth=2)
#     ax.plot(ts_ds.index, ts_ds[col].values, linewidth=2)

#     ax.set_title(f'Fold {fold} - {col}', fontsize=fsize_title)
#     ax.set_xlabel('Time', fontsize=fsize_labels)
#     ax.set_ylabel('Value', fontsize=fsize_labels)
#     ax.tick_params(labelsize=fsize_ticks)
#     ax.grid(False)
#     plt.tight_layout()

#     plt.savefig(f'{data_path}/SUPERVISED_DATASETS/{fold}_{cut_col_name(col)}_data.jpg')

#     plt.show()

In [ ]:
def create_supervised_dataset(dataset, input_length, output_length):
  X = []
  Y = []
  rows = dataset.shape[0]
  for i in range(rows - input_length - output_length + 1):
    X.append(dataset[i:i + input_length])
    Y.append(dataset[i + input_length:i + input_length + output_length])
  return np.array(X), np.array(Y)

In [ ]:
tr_datasets = {}
vl_datasets = {}
ts_datasets = {}

INPUT_LENGTH = 1
OUTPUT_LENGTH = 1

for fold, dataset in datasets.items():
    tr, vl, ts = dataset
    print('--' * 20)
    print(f'Iteration {fold}: {tr.shape}, {vl.shape}, {ts.shape}')
    print('--' * 20)

    tr_datasets[fold] = create_supervised_dataset(tr.values, INPUT_LENGTH, OUTPUT_LENGTH)
    vl_datasets[fold] = create_supervised_dataset(vl.values, INPUT_LENGTH, OUTPUT_LENGTH)
    ts_datasets[fold] = create_supervised_dataset(ts.values, INPUT_LENGTH, OUTPUT_LENGTH)

    print('Input (BATCHES x INPUT_LENGTH x FEATURES) and output sizes (BATCHES x OUTPUT_LENGTH x FEATURES)')
    print(f'Train Set - x_tr: {tr_datasets[fold][0].shape}, y_tr: {tr_datasets[fold][1].shape}')
    print(f'Validation Set - x_vl: {vl_datasets[fold][0].shape}, y_vl: {vl_datasets[fold][1].shape}')
    print(f'Test Set - x_ts: {ts_datasets[fold][0].shape}, y_ts: {ts_datasets[fold][1].shape}')


In [ ]:
def escalar_dataset(data_input):
    NFEATS = data_input['x_tr'].shape[2]

    # Generate list with "scalers" (1 for each input covariate)
    scalers = [MinMaxScaler(feature_range=(-1,1)) for i in range(NFEATS)]
    print(len(scalers), NFEATS)

    # Arreglos que contendrán los datasets escalados
    x_tr_s = np.zeros(data_input['x_tr'].shape)
    x_vl_s = np.zeros(data_input['x_vl'].shape)
    x_ts_s = np.zeros(data_input['x_ts'].shape)
    y_tr_s = np.zeros(data_input['y_tr'].shape)
    y_vl_s = np.zeros(data_input['y_vl'].shape)
    y_ts_s = np.zeros(data_input['y_ts'].shape)

    x_tr_ds, y_tr_ds = data_input['x_tr'], data_input['y_tr']
    x_val_ds, y_val_ds = data_input['x_vl'], data_input['y_vl']
    x_ts_ds, y_ts_ds = data_input['x_ts'], data_input['y_ts']

    # Scaling: the min/max of the training set will be used to scale all the datasets.

    # Scaling Xs
    for i in range(NFEATS):
        x_tr_s[:,:,i] = scalers[i].fit_transform(x_tr_ds[:,:,i])
        x_ts_s[:,:,i] = scalers[i].transform(x_ts_ds[:,:,i])
        x_vl_s[:,:,i] = scalers[i].transform(x_val_ds[:,:,i])

        y_tr_s[:,:,i] = scalers[i].transform(y_tr_ds[:,:,i])
        y_vl_s[:,:,i] = scalers[i].transform(y_val_ds[:,:,i])
        y_ts_s[:,:,i] = scalers[i].transform(y_ts_ds[:,:,i])


    # Create output dictionary
    data_scaled = {
        'x_tr_s': x_tr_s, 'y_tr_s': y_tr_s,
        'x_vl_s': x_vl_s, 'y_vl_s': y_vl_s,
        'x_ts_s': x_ts_s, 'y_ts_s': y_ts_s,
    }

    return data_scaled, scalers

In [ ]:
tr_s_ds, vl_s_ds, ts_s_ds = {}, {}, {}
scalers = {}

index = 0

for (x_tr_ds, y_tr_ds), (x_val_ds, y_val_ds), (x_ts_ds, y_ts_ds) in zip(tr_datasets.values(), vl_datasets.values(), ts_datasets.values()):

    print('--' * 20)
    print(f'Iteration {index}: {x_tr_ds.shape}, {x_val_ds.shape}, {x_ts_ds.shape}')
    print('--' * 20)

    data_in = {
        'x_tr': x_tr_ds, 'y_tr': y_tr_ds,
        'x_vl': x_val_ds, 'y_vl': y_val_ds,
        'x_ts': x_ts_ds, 'y_ts': y_ts_ds,
    }

    # Y scalar (specifying the column with the variable to predict)
    data_s, scalers_result = escalar_dataset(data_in,)

    tr_s_ds[index] = data_s['x_tr_s'], data_s['y_tr_s']
    vl_s_ds[index] = data_s['x_vl_s'], data_s['y_vl_s']
    ts_s_ds[index] = data_s['x_ts_s'], data_s['y_ts_s']
    scalers[index] = scalers_result

    index += 1

In [ ]:
# And let's generate a violin plot to see the distribution of values in each covariate (input) and in the variable to predict (output).
# index = 0
# for (x_tr_s, y_tr_s), (x_vl_s, y_vl_s), (x_ts_s, y_ts_s) in zip(tr_s_ds.values(), vl_s_ds.values(), ts_s_ds.values()):
#   print('--' * 20)
#   print(f'Fold {index}: {x_tr_s.shape}, {x_vl_s.shape}, {x_ts_s.shape}')
#   print('--' * 20)
#   index += 1
#   for i, col in enumerate(df.columns):
#     plt.figure(figsize=(10, 8))

#     plt.subplot(1, 1, 1)
#     plt.violinplot(dataset=x_tr_s[:,:,i].flatten(), positions=[0])
#     plt.violinplot(dataset=x_vl_s[:,:,i].flatten(), positions=[0])
#     plt.violinplot(dataset=x_ts_s[:,:,i].flatten(), positions=[0])
#     plt.title(f'Fold {index} - {col}', fontdict={'fontsize': 20})

#     plt.xticks(fontsize=20)
#     plt.yticks(fontsize=20)

#     plt.grid(False)
#     plt.savefig(f'{data_path}/DATA_STATISTICS/fold-{index}_{cut_col_name(col)}_violin.jpg')
#     # plt.savefig(f'{col.replace("/", "_")}_frecuencia.png')
#     plt.show()

# 2. BI-GRU Model Building

In [ ]:
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, MeanSquaredError

In [ ]:
#gpus = tf.config.list_physical_devices('GPU')
#if gpus:
#  try:
#    # Currently, memory growth needs to be the same across GPUs
#    for gpu in gpus:
#      tf.config.experimental.set_memory_growth(gpu, True)
#    logical_gpus = tf.config.list_logical_devices('GPU')
#    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#  except RuntimeError as e:
#    # Memory growth must be set before GPUs have been initialized
#    print(e)

In [ ]:
dict_variables_results = {}

for i, col in enumerate(df.columns):
  dict_variables_results[col] = {
      "rmse": 0,
      "mae": 0,
      "mape": 0,
  }

In [ ]:
# import libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Flatten, Conv1D, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam, Adagrad
from keras.regularizers import l2

from tensorflow.keras.utils import plot_model
from scipy.stats import shapiro
from time import time


In [ ]:
# Model creation


# Loss: the RMSE (root mean squared error) will be used for training because it allows to have errors in the same units as each variable.
def root_mean_squared_error(y_true, y_pred):
    rmse = tf.math.sqrt(tf.math.reduce_mean(tf.square(y_pred-y_true)))
    return rmse

font_size_ticks = 20
mae = MeanAbsoluteError()
mape = MeanAbsolutePercentageError()

for jp in range(10):
    
    index = 0
    
    print('RUN N°', jp)

    for (x_tr_s, y_tr_s), (x_vl_s, y_vl_s), (x_ts_s, y_ts_s) in zip(tr_s_ds.values(), vl_s_ds.values(), ts_s_ds.values()):

      print('--' * 20)
      print(f'Iteration {index}: {x_tr_s.shape}, {x_ts_s.shape}')
      print('--' * 20)
    
      scaler = scalers[index]
      tr_cut_date, vl_cut_date, ts_cut_date = cut_dates[index]
    
      tr, vl, ts = datasets[index]
      x_ts, y_true = ts_datasets[index]
    
      for i, col in enumerate(df.columns):
    
        start = time()
    
        print(f'Training BI-GRU... for {col}')
    
        model = Sequential([
            Bidirectional(GRU(128, input_shape=(x_tr_ds.shape[1], 1), return_sequences = True)),
            Bidirectional(GRU(64, return_sequences = False)),
            Dense(1, activation='linear')
        ])
    
    
        model.compile(optimizer = RMSprop(5e-5), loss = root_mean_squared_error, metrics = [MeanAbsoluteError(), MeanAbsolutePercentageError()])
    
        # Training (approximately 1 min using GPU)
    
        EPOCHS = 250
        BATCH_SIZE = 256

        story = model.fit(
            x = np.expand_dims(x_tr_s[:,:,i], axis=-1),
            y = np.expand_dims(y_tr_s[:,:,i], axis=-1),
            batch_size = BATCH_SIZE,
            epochs = EPOCHS,
            validation_data = (np.expand_dims(x_vl_s[:,:,i], axis=-1), np.expand_dims(y_vl_s[:,:,i], axis=-1)),
            verbose=2,
            callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
        )
    
        finish = time() - start

        plt.figure(figsize=(15, 7))
        plt.subplot(1, 1, 1)
        plt.plot(story.history['loss'], label='RMSE train', linewidth=line_width)
        plt.plot(story.history['val_loss'],label='RMSE test', linewidth=line_width)
        #plt.title(f'Iteration {index} - {col}', fontsize=28)
        plt.xlabel('Epoch', fontsize=fsize_labels)
        plt.ylabel('RMSE', fontsize=fsize_labels)
        plt.xticks(fontsize=font_size_ticks)
        plt.yticks(fontsize=font_size_ticks)
        plt.tight_layout()
        plt.legend(fontsize=25);
    
        plt.grid(False)
        plt.savefig(f'{data_path}/BiGru/metrics_result/iteration_{index}_{col.replace("/", "_")}_rmse.pdf')
        plt.tight_layout()
        plt.show()
    
        plt.figure(figsize=(15, 7))
        plt.subplot(1, 1, 1)
        plt.plot(story.history['mean_absolute_error'], label='MAE train', linewidth=line_width)
        plt.plot(story.history['val_mean_absolute_error'],label='MAE test', linewidth=line_width)
        #plt.title(f'Iteration {index} - {col}',  fontsize=28)
        plt.xlabel('Epoch', fontsize=fsize_labels)
        plt.ylabel('MAE', fontsize=fsize_labels)
        plt.xticks(fontsize=font_size_ticks)
        plt.yticks(fontsize=font_size_ticks)
        plt.tight_layout()
        plt.legend(fontsize=25);
    
        plt.grid(False)
        plt.savefig(f'{data_path}/BiGru/metrics_result/Iteration_{index}_{col.replace("/", "_")}_mae.pdf')
        plt.tight_layout()
        plt.show()

        plt.figure(figsize=(15, 7))
        plt.subplot(1, 1, 1)
        plt.plot(story.history['mean_absolute_percentage_error'], label='MAPE train', linewidth=line_width)
        plt.plot(story.history['val_mean_absolute_percentage_error'],label='MAPE test', linewidth=line_width)
        #plt.title(f'Iteration {index} - {col}', fontsize=28)
        plt.xlabel('Epoch', fontsize=fsize_labels)
        plt.ylabel('MAPE', fontsize=fsize_labels)
        plt.xticks(fontsize=font_size_ticks)
        plt.yticks(fontsize=font_size_ticks)
        plt.tight_layout()
        plt.legend(fontsize=25);

        plt.grid(False)
        plt.savefig(f'{data_path}/BiGru/metrics_result/iteration_{index}_{col.replace("/", "_")}_mape.pdf')
        plt.tight_layout()
        plt.show()
    
        story = None

        y_ts_predict = scaler[i].inverse_transform(model.predict(x_ts_s[:,:,i], verbose=0))
    
        ts_predict = pd.DataFrame(y_ts_predict[:, 0], index=ts[:-1].index)
    
        mae = MeanAbsoluteError()
        mape = MeanAbsolutePercentageError()
    
        mae.update_state(y_true[:, :, i], y_ts_predict)
        mape.update_state(y_true[:, :, i], y_ts_predict)

        i2, b2 = y_ts_predict.shape

    # print(f'VARIABLE ==> [{col}]\n')
    # print(f'RMSE -> {round(root_mean_squared_error(y_true[:, :, i], y_ts_predict).numpy(), 3)} \n')
    # print(f'MAE ->  {round(mae.result().numpy(), 3)} \n')
    # print(f'MAPE -> { round(mape.result().numpy(), 3)} \n')
    # print(f'SW statistic: -> {shapiro(abs(y_ts_predict.reshape(i2, b2))).statistic} \n')
    # print(f'TRAIN TIME -> {round(finish, 3)} \n')


        with open(f'{data_path}/BiGru/metrics_result/{jp}_bi-gru_Iteration_{index}_results_ts.txt', 'a') as file:
          file.write(f'VARIABLE ; [{col}]\n')
          file.write(f'RMSE ; {round(root_mean_squared_error(y_true[:, :, i], y_ts_predict).numpy(), 3)} \n')
          file.write(f'MAE ;  {round(mae.result().numpy(), 3)} \n')
          file.write(f'MAPE ; { round(mape.result().numpy(), 3)} \n')
    
          #result = (abs(y_ts_predict.reshape(i2, b2)))
          #file.write(f'SW statistic: -> {result.statistic}, p-value: {result.pvalue} \n')
    
          file.write(f'SW statistic: ; {shapiro(abs(y_ts_predict.reshape(i2, b2))).statistic} \n')
          file.write(f'TRAIN TIME ; {round(finish, 3)} \n')
            

        plt.figure(figsize=(24, 10))
        plt.subplot(1, 1, 1)
        plt.plot(ts.index, ts[[col]].values,  label='Real', linewidth=line_width)
        plt.plot(ts_predict.index, ts_predict.values, label='Predicted BiGRU', linewidth=line_width)
        plt.title(f'Iteration {index} - {col}', fontdict={'fontsize': 30})
        plt.xlabel('Time', fontsize=fsize_labels*2)
        plt.ylabel('Value', fontsize=fsize_labels*2)

        plt.xticks(fontsize=font_size_ticks*2)
        plt.yticks(fontsize=font_size_ticks*2)
        plt.tight_layout()
    
        plt.grid(False)
        plt.legend(fontsize=25)
        plt.savefig(f'{data_path}/BiGru/test_result/iteration_{index}_{col.replace("/", "_")}_test_result.png')
        plt.show()

        fig, axe = plt.subplots(figsize=(5, 7))
        box = axe.boxplot(ts_predict.values.shape - ts[[col]].values[:-1], notch=True, vert=True, autorange=True, sym="")
        for box in box['boxes']:
          box.set_color('b')
          axe.set_title(f'Iteration {index} - {col}', fontsize=22)
          axe.tick_params(labelsize=20)
          plt.tight_layout()
          plt.savefig(f'{data_path}/BiGru/test_result/iteration-{index}_{cut_col_name(col)}_test_result_box_plot.png')
        # break
      # break
      index += 1

# 3. LSTM Model Building

In [ ]:
#!nvidia-smi

In [ ]:
# Loss: the RMSE (root mean squared error) will be used for training because it allows to have errors in the same units as each variable.
def root_mean_squared_error(y_true, y_pred):
    rmse = tf.math.sqrt(tf.math.reduce_mean(tf.square(y_pred-y_true)))
    return rmse

In [ ]:
dict_variables_results = {}

for i, col in enumerate(df.columns):
  dict_variables_results[col] = {
      "rmse": 0,
      "mae": 0,
      "mape": 0,
  }

In [ ]:
# Model creation

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Flatten, Conv1D, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam, Adagrad
from keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from scipy.stats import shapiro
from time import time

# Loss: the RMSE (root mean squared error) will be used for training because it allows to have errors in the same units as each variable.
def root_mean_squared_error(y_true, y_pred):
    rmse = tf.math.sqrt(tf.math.reduce_mean(tf.square(y_pred-y_true)))
    return rmse

for jp in range(10):

  index = 0

  print('RUN N°', jp)

  for (x_tr_s, y_tr_s), (x_vl_s, y_vl_s), (x_ts_s, y_ts_s) in zip(tr_s_ds.values(), vl_s_ds.values(), ts_s_ds.values()):

    print('--' * 20)
    print(f'Iteration {index}: {x_tr_s.shape}, {x_ts_s.shape}')
    print('--' * 20)

    scaler = scalers[index]
    tr_cut_date, vl_cut_date, ts_cut_date = cut_dates[index]

    tr, vl, ts = datasets[index]
    x_ts, y_true = ts_datasets[index]

    for i, col in enumerate(df.columns):

      start = time()

      print(f'Training LSTM... for {col}')

      model = Sequential([
          LSTM(128, input_shape=(x_tr_ds.shape[1], 1), return_sequences = True,),
          Dropout(0.1),
          LSTM(128, return_sequences = True),
          LSTM(64, return_sequences = False),
          Dense(16, activation='relu'),
          Dense(1, activation='linear')]
      )


      model.compile(optimizer = RMSprop(5e-5), loss = root_mean_squared_error, metrics = [MeanAbsoluteError(), MeanAbsolutePercentageError()])

      # Training (approximately 1 min using GPU)

      EPOCHS = 250
      BATCH_SIZE = 256

      story = model.fit(
          x = np.expand_dims(x_tr_s[:,:,i], axis=-1),
          y = np.expand_dims(y_tr_s[:,:,i], axis=-1),
          batch_size = BATCH_SIZE,
          epochs = EPOCHS,
          validation_data = (np.expand_dims(x_vl_s[:,:,i], axis=-1), np.expand_dims(y_vl_s[:,:,i], axis=-1)),
          verbose=2,
          callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],
      )

      finish = time() - start

      plt.figure(figsize=(15, 7))
      plt.subplot(1, 1, 1)
      plt.plot(story.history['loss'], label='RMSE train', linewidth=line_width)
      plt.plot(story.history['val_loss'],label='RMSE val', linewidth=line_width)
      plt.title(f'Iteration {index} - {col}', fontsize=28)
      plt.xlabel('Epoch', fontsize=20)
      plt.ylabel('RMSE', fontsize=20)
      plt.xticks(fontsize=font_size_ticks)
      plt.yticks(fontsize=font_size_ticks)
      plt.tight_layout()
      plt.legend(fontsize=22);

      plt.grid(False)
      plt.savefig(f'{data_path}/LSTM/metrics_result/iteration_{index}_{col.replace("/", "_")}_rmse.png')
      plt.tight_layout()
      plt.show()

      plt.figure(figsize=(15, 7))
      plt.subplot(1, 1, 1)
      plt.plot(story.history['mean_absolute_error'], label='MAE train', linewidth=line_width)
      plt.plot(story.history['val_mean_absolute_error'],label='MAE val', linewidth=line_width)
      plt.title(f'Iteration {index} - {col}',  fontsize=28)
      plt.xlabel('Epoch', fontsize=20)
      plt.ylabel('MAE', fontsize=20)
      plt.xticks(fontsize=font_size_ticks)
      plt.yticks(fontsize=font_size_ticks)
      plt.tight_layout()
      plt.legend(fontsize=22);

      plt.grid(False)
      plt.savefig(f'{data_path}/LSTM/metrics_result/iteration_{index}_{col.replace("/", "_")}_mae.png')
      plt.tight_layout()
      plt.show()

      plt.figure(figsize=(15, 7))
      plt.subplot(1, 1, 1)
      plt.plot(story.history['mean_absolute_percentage_error'], label='MAPE train', linewidth=line_width)
      plt.plot(story.history['val_mean_absolute_percentage_error'],label='MAPE val', linewidth=line_width)
      plt.title(f'Iteration {index} - {col}', fontsize=28)
      plt.xlabel('Epoch', fontsize=20)
      plt.ylabel('MAPE', fontsize=20)
      plt.xticks(fontsize=font_size_ticks)
      plt.yticks(fontsize=font_size_ticks)
      plt.tight_layout()
      plt.legend(fontsize=22);

      plt.grid(False)
      plt.savefig(f'{data_path}/LSTM/metrics_result/iteration_{index}_{col.replace("/", "_")}_mape.png')
      plt.tight_layout()
      plt.show()

      story = None

      y_ts_predict = scaler[i].inverse_transform(model.predict(x_ts_s[:,:,i], verbose=0))

      ts_predict = pd.DataFrame(y_ts_predict[:, 0], index=ts[:-1].index)

      mae = MeanAbsoluteError()
      mape = MeanAbsolutePercentageError()

      mae.update_state(y_true[:, :, i], ts_predict.values)
      mape.update_state(y_true[:, :, i], ts_predict.values)

      i2, b2 = y_ts_predict.shape

      # with open(f'./lstm-fold_{index}_results_ts.txt', 'a') as file:
      with open(f'{data_path}/LSTM/metrics_result/{jp}_Iteration_{index}_results_ts.txt', 'a') as file:
        file.write(f'VARIABLE ==> [{col}]\n')
        file.write(f'RMSE -> {round(root_mean_squared_error(y_true[:, :, i], ts_predict.values).numpy(), 3)} \n')
        file.write(f'MAE ->  {round(mae.result().numpy(), 3)} \n')
        file.write(f'MAPE -> { round(mape.result().numpy(), 3)} \n')
        file.write(f'SW statistic: -> {shapiro(abs(y_ts_predict.reshape(i2, b2))).statistic} \n')
        file.write(f'TRAIN TIME -> {round(finish, 3)} \n')

      plt.figure(figsize=(24, 10))
      plt.subplot(1, 1, 1)
      plt.plot(ts.index, ts[[col]].values, label='Real', linewidth=line_width)
      plt.plot(ts_predict.index, ts_predict.values, label='Predicted LSTM', linewidth=line_width)
      plt.title(f'Iteration {index} - {col}', fontdict={'fontsize': 28})
      plt.xlabel('Time', fontsize=20)
      plt.ylabel('Value', fontsize=20)

      plt.xticks(fontsize=font_size_ticks)
      plt.yticks(fontsize=font_size_ticks)
      plt.grid(False)
      plt.legend(fontsize=22)
      plt.savefig(f'{data_path}/LSTM/test_result/iteration_{index}_{col.replace("/", "_")}_test_result.png')
      plt.show()

      fig, axe = plt.subplots(figsize=(5, 7))
      box = axe.boxplot(ts_predict.values - ts[[col]][:-1].values, notch=True, vert=True, autorange=True, sym="")
      for box in box['boxes']:
        box.set_color('b')
        axe.set_title(f'Iteration {index} - {col}', fontsize=23)
        axe.tick_params(labelsize=23)
        plt.tight_layout()
        plt.savefig(f'{data_path}/LSTM/test_result/iteration-{index}_{cut_col_name(col)}_test_result_box_plot.jpg')
      # break

    index += 1

# 3. ARIMA Model Building

In [ ]:
!pip install pmdarima

In [ ]:
def metrics_values(y_true, y_pred):

  for index, column in enumerate(tr.columns):
    mae = MeanAbsoluteError()
    mape = MeanAbsolutePercentageError()
    mae.update_state(y_true[:, :, index], y_pred[:, :, index])
    mape.update_state(y_true[:, :, index], y_pred[:, :, index])

    with open('results_tr.txt', 'a') as file:
      file.write(f'VARIABLE;[{column}]\n')
      file.write(f'RMSE;{round(root_mean_squared_error(y_true[:, :, index], y_pred[:, :, index]).numpy(), 3)} \n')
      file.write(f'MAE;{round(mae.result().numpy(), 3)} \n')
      file.write(f'MAPE;{ round(mape.result().numpy(), 3)} \n')
    # file.write(f'MASE -> { round(mase.result().numpy(), 3)} \n')

In [ ]:
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, MeanSquaredError
from pmdarima import ARIMA, auto_arima
from scipy.stats import shapiro

In [ ]:
def root_mean_squared_error(y_true, y_pred):
    rmse = tf.math.sqrt(tf.math.reduce_mean(tf.square(y_pred-y_true)))
    return rmse

In [ ]:
from time import time

#index = 0
print("ARIMA")

#dict_variables = {}

#dict_variables_results = {}

#for i, col in enumerate(df.columns):
#  dict_variables_results[col] = {
#      "rmse": 0,
#      "mae": 0,
#      "mape": 0,
#  }

font_size_ticks = 20

for jp in range(10):
  
    index = 0
    
    print('RUN N°', jp)
    
    for (x_tr_s, y_tr_s), (x_vl_s, y_vl_s), (x_ts_s, y_ts_s) in zip(tr_s_ds.values(), vl_s_ds.values(), ts_s_ds.values()):
    
        print('--' * 20)
        print(f'Training ARIMA... for {col} {x_tr_s[:,:,i].shape} ---- {tr.index[:-1].shape}')
        print('--' * 20)
    
        scaler = scalers[index]
        tr_cut_date, vl_cut_date, ts_cut_date = cut_dates[index]
    
        tr, vl, ts = datasets[index]
        x_ts, y_true = ts_datasets[index]
    
        for i, col in enumerate(df.columns):
          start = time()
    
          print(f'Training ARIMA... for {col}')
    
          #modelo.summary()
    
          modelo = ARIMA(order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
          modelo.fit(y=pd.DataFrame(x_tr_s[:,:,i], index=tr.index[:-1])) # tomo hasta la penultima fecha
    
          #predicciones_pdmarima = modelo.predict(len(x_vl_s.shape[0]) + len(x_ts_s.shape[0]))
    
          predicciones_pdmarima = modelo.predict(len(vl) + len(ts))
          predicciones_pdmarima.name = 'predicciones_pdmarima'
          pred_arima = pd.DataFrame(predicciones_pdmarima[predicciones_pdmarima.index > str(vl_cut_date)])
    
          plt.figure(figsize=(18, 15))
          plt.subplot(2, 1, 1)
    
          scaled_predicted = scaler[i].inverse_transform(pred_arima.values)
    
          finish = time() - start
    
          # pd.DataFrame({'predict': scaler[i].inverse_transform(pred_arima.values).squeeze(), 'real': ts[[col]].values.squeeze()}, index=ts.index).to_csv('result')
    
          plt.plot(ts.index[:-1], ts[[col]][:-1], label='Real', linewidth=line_width)
          plt.plot(pred_arima.index, scaled_predicted, label='Estimated ARIMA - pmdarima', linewidth=line_width)
          plt.title(f'Iteration {index} - {col}', fontdict={'fontsize': 28})
          plt.legend(fontsize=25)
          plt.xlabel('Time', fontsize=30)
          plt.ylabel('Value', fontsize=30)
    
          plt.xticks(fontsize=font_size_ticks) # aumento de datos
          plt.yticks(fontsize=font_size_ticks)
    
          plt.grid(False)
          plt.savefig(f'{data_path}/Arima/test_result/Iteration-{index}_{cut_col_name(col)}_test_result.png')
          plt.tight_layout()
          plt.show()
    
    #      story = None
            
          mae = MeanAbsoluteError()
          mape = MeanAbsolutePercentageError()
    
          mae.update_state(y_true[:, :, i], scaled_predicted)
          mape.update_state(y_true[:, :, i], scaled_predicted)
    
          i2, b2 = scaled_predicted.shape
    
          with open(f'{data_path}/Arima/metrics_result/{jp}_Arima_Iteration_{index}_results_ts.txt', 'a') as file:
          # with open(f'./AutoArima-fold_{index}_results_ts.txt', 'a') as file:
            file.write(f'VARIABLE;[{col}]\n')
            file.write(f'RMSE;{round(root_mean_squared_error(y_true[:, :, i], scaled_predicted).numpy(), 3)} \n')
            file.write(f'MAE;{round(mae.result().numpy(), 3)} \n')
            file.write(f'MAPE;{ round(mape.result().numpy(), 3)} \n')
            file.write(f'SW statistic;{shapiro(abs(scaled_predicted.reshape(i2, b2))).statistic} \n')
            file.write(f'TRAIN TIME;{round(finish, 3)} \n')
    
          dict_variables_results[col]['rmse'] = dict_variables_results[col]['rmse'] + round(root_mean_squared_error(y_true[:, :, i], scaled_predicted).numpy(), 3)
          dict_variables_results[col]['mae'] = dict_variables_results[col]['mae'] + round(mae.result().numpy(), 3)
          dict_variables_results[col]['mape'] = dict_variables_results[col]['mape'] + round(mape.result().numpy(), 3)
    
          fig, axe = plt.subplots(figsize=(5, 7))
          box = axe.boxplot(scaled_predicted - ts[[col]][:-1], notch=True, vert=True, autorange=True, sym="")
          for box in box['boxes']:
            box.set_color('b')
            axe.set_title(f'Iteration {index} - {col}', fontsize=23)
            axe.tick_params(labelsize=23)
            plt.tight_layout()
            plt.savefig(f'{data_path}/Arima/test_result/Iteration-{index}_{cut_col_name(col)}_test_result_box_plot.png')
    
        
        plt.show()
        # break
    
    
        for col in df.columns:
            with open(f'{data_path}/Arima/metrics_result/{jp}_Iterations_results_summary.txt', 'a') as file:
              file.write(f'VARIABLE; [{col}]\n')
              file.write(f'RMSE; {dict_variables_results[col]["rmse"]} \n')
              file.write(f'MAE; {dict_variables_results[col]["mae"]} \n')
              file.write(f'MAPE; {dict_variables_results[col]["mape"]} \n')
    
        index += 1

# 4. AUTO ARIMA Model Building

In [ ]:
# from time import time

index = 0

dict_variables = {}

dict_variables_results = {}

for i, col in enumerate(df.columns):
  dict_variables_results[col] = {
      "rmse": 0,
      "mae": 0,
      "mape": 0,
      "time": 0,
  }


for jp in range(1):

  for (x_tr_s, y_tr_s), (x_vl_s, y_vl_s), (x_ts_s, y_ts_s) in zip(tr_s_ds.values(), vl_s_ds.values(), ts_s_ds.values()):

    print('--' * 20)
    print(f'Training ARIMA... for {col} {x_tr_s[:,:,i].shape} ---- {tr.index[:-1].shape}')
    print('--' * 20)

    scaler = scalers[index]
    tr_cut_date, vl_cut_date, ts_cut_date = cut_dates[index]

    tr, vl, ts = datasets[index]
    x_ts, y_true = ts_datasets[index]

    for i, col in enumerate(df.columns):
      start = time()

      print(f'Training AUTOARIMA... for {col}')

      modelo = auto_arima(pd.DataFrame(x_tr_s[:,:,i], index=tr.index[:-1]),
                        m=12,               # frequency of series
                        d=None,             # let model determine 'd'
                        test='adf',         # use adftest to find optimal 'd'
                        start_p=0, start_q=0, # minimum p and q
                        max_p=12, max_q=12, # maximum p and q
                        D=None,             # let model determine 'D'
                        trace=True,
                        error_action='ignore',
                        suppress_warnings=True,
                        stepwise=True,
      )

      modelo.summary()


      predicciones_pdmarima = modelo.predict(len(vl) + len(ts))
      predicciones_pdmarima.name = 'predicciones_pdmarima'
      pred_arima = pd.DataFrame(predicciones_pdmarima[predicciones_pdmarima.index > str(vl_cut_date)])

      plt.figure(figsize=(18, 15))
      plt.subplot(2, 1, 1)

      scaled_predicted = scaler[i].inverse_transform(pred_arima.values)

      finish = time() - start

      # pd.DataFrame({'predict': scaler[i].inverse_transform(pred_arima.values).squeeze(), 'real': ts[[col]].values.squeeze()}, index=ts.index).to_csv('result')

      plt.plot(ts.index[:-1], ts[[col]][:-1], label='Real', linewidth=line_width)
      plt.plot(pred_arima.index, scaled_predicted, label='Predicted ARIMA - pmdarima', linewidth=line_width)
      plt.title(f'Iteration {index} - {col}', fontdict={'fontsize': 28})
      plt.legend(fontsize=22)

      plt.xticks(fontsize=20) # aumento de datos
      plt.yticks(fontsize=20)

      plt.grid(False)
      plt.savefig(f'{data_path}/AutoArima/test_result/Iteration-{index}_{cut_col_name(col)}_test_result.jpg')
      plt.tight_layout()
      plt.show()

      mae = MeanAbsoluteError()
      mape = MeanAbsolutePercentageError()

      mae.update_state(y_true[:, :, i], scaled_predicted)
      mape.update_state(y_true[:, :, i], scaled_predicted)

      i2, b2 = scaled_predicted.shape

      with open(f'{data_path}/AutoArima/metrics_result/{jp}_Iteration_{index}_results_ts.txt', 'a') as file:
      # with open(f'./AutoArima-fold_{index}_results_ts.txt', 'a') as file:
        file.write(f'VARIABLE ==> [{col}]\n')
        file.write(f'RMSE -> {round(root_mean_squared_error(y_true[:, :, i], scaled_predicted).numpy(), 3)} \n')
        file.write(f'MAE ->  {round(mae.result().numpy(), 3)} \n')
        file.write(f'MAPE -> { round(mape.result().numpy(), 3)} \n')
        file.write(f'SW statistic: -> {shapiro(abs(scaled_predicted.reshape(i2, b2))).statistic} \n')
        file.write(f'TRAIN TIME -> {round(finish, 3)} \n')

      dict_variables_results[col]['rmse'] = dict_variables_results[col]['rmse'] + round(root_mean_squared_error(y_true[:, :, i], scaled_predicted).numpy(), 3)
      dict_variables_results[col]['mae'] = dict_variables_results[col]['mae'] + round(mae.result().numpy(), 3)
      dict_variables_results[col]['mape'] = dict_variables_results[col]['mape'] + round(mape.result().numpy(), 3)
      dict_variables_results[col]['time'] = dict_variables_results[col]['time'] + round(finish, 3)

      fig, axe = plt.subplots(figsize=(5, 7))
      box = axe.boxplot(scaled_predicted - ts[[col]][:-1], notch=True, vert=True, autorange=True, sym="")
      for box in box['boxes']:
        box.set_color('b')
        axe.set_title(f'Iteration {index} - {col}', fontsize=23)
        axe.tick_params(labelsize=23)
        plt.tight_layout()
        plt.savefig(f'{data_path}/AutoArima/test_result/Iteration-{index}_{cut_col_name(col)}_test_result_box_plot.jpg')

    index += 1

    plt.show()
    # break


  for col in df.columns:
    with open(f'{data_path}/AutoArima/metrics_result/{jp}_Iterations_results_summary.txt', 'a') as file:
      file.write(f'VARIABLE , [{col}]\n')
      file.write(f'RMSE , {dict_variables_results[col]["rmse"]} \n')
      file.write(f'MAE , {dict_variables_results[col]["mae"]} \n')
      file.write(f'MAPE , {dict_variables_results[col]["mape"]} \n')
      file.write(f'TRAIN TIME , {dict_variables_results[col]["time"]} \n')  

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')